In [1]:
"""
    pow(number, power, base)
"""
function pow(number::Signed, power::Signed, base::Signed)::BigInt
    power = digits(power, base=2);
    result = BigInt(1);
    for i = length(power):-1:1
        result = BigInt(result^2 * number^power[i]) % base;
    end;

    return result;
end

function st(n::Integer)
    for s=1:BigInt(floor(log2(n)))
        t, r = divrem(n, 2^s);
        r == 0 && isodd(t) && return (s, t);
    end
    
    throw(error("Wrong number pattern"));
end

"""
    witness(a, n)

Test for evidence of simplicity.
"""
function witness(a::Integer, n::Integer)
    if gcd(a, n) > 1
        return false;
    end
    
    s, t = st(n-1);    
    b = pow(BigInt(a), t, n);
    
    if b == 1 || b == n-1
        return true;
    end
    
    for i=1:s-1
        b = pow(b, 2, n);
        if b == n-1
            return true;
        end
    end
    
    return false;
end

"""
    mrpt(n, k)

Miller–Rabin primality test
"""
function mrpt(n::Integer, k::Int=1)
    for i=1:k
        a = rand(2:n-1);
        if !witness(a, n)
            return false;
        end
    end
    
    return true;
end

mrpt

# Математические основы защиты информации
# Лабораторная работа №5
# Факторизация целых чисел (экспоненциальные методы)

## Задание 1.
Черепаха с генератором случайных чисел

In [2]:
"""
    randomTurtle(n::Int)

n - composite number

Returns: non-trivial divisor of n
"""
function randomTurtle(n::Integer)
    mrpt(n) && throw("n is prime")
    
    x = [rand(0:n-1)]
    
    i = 1;
    while true
        i += 1;
        push!(x, rand(0:n-1));
        
        for j=1:i-1
            d = gcd(x[i] - x[j], n);
            1 < d && d < n && return d;
        end;
    end;
end

randomTurtle

In [3]:
randomTurtle(10000)

ErrorException: [91mWrong number pattern[39m

## Задание 2.
Черепаха

In [4]:
"""
    Turtle(n::Int, a::Int=1, x0::Int=1)

n - composite number

Returns: non-trivial divisor of n
"""
function Turtle(n::Integer, a::Int=1, x0::Integer=1)
    f(x::Integer=x0) = (x^2 + a) % n; 
    
    x = [x0];
    
    i = 1;
    while true
        i += 1;
        push!(x, f(x[end]));
        
        for j=1:i-1
            d = gcd(x[i] - x[j], n);
            1 < d && d < n && return d;
            d == n && return "Choose other x0";
        end;
    end;
end

Turtle

In [5]:
Turtle(100)

4

## Задание 3.
Черепаха и заяц

In [6]:
"""
    TurtleHare(n::Int, a::Int=1, x0::Int=1)

n - composite number

Returns: non-trivial divisor of n
"""
function memoriezedTurtleHare(n::Integer, a::Int=1, x0::Integer=1)
    f(x::Integer=x0) = (x^2 + a) % n; 
    
    x = Dict([0 => x0]);
    
    i = 0;
    while true
        i += 1;
        x[2i-1] = f(x[2i-2]);
        x[2i] = f(x[2i-1]);
        
        d = gcd(x[2i] - x[i], n);
        1 < d && d < n && return d;
        d == n && return "Choose other x0";
    end;
end

memoriezedTurtleHare

In [7]:
memoriezedTurtleHare(100)

4

## Задание 4.

In [8]:
n1 = 250555579698426283;
n2 = 41220545481281889966969492498051061589102446563174388598626134746729206699882766512826923833210650429764673812076262586428127165108100437049590467499913625296023687050676867783808603120181062808207599561741;

In [9]:
@time randomTurtle(n1) # слишком медленный, т. к. рандомно числа выбираются

 13.312041 seconds (1.20 k allocations: 279.375 KiB)


113493047

In [10]:
@time Turtle(n1) # 

  1.447628 seconds (13 allocations: 64.547 KiB)


113493047

In [11]:
@time memoriezedTurtleHare(n1) # медленне предыдущего, т. к. словарь медленне массива и много операций

  4.616585 seconds (80 allocations: 1.060 GiB, 3.00% gc time)


2207673389

In [12]:
@time Turtle(n2, 1, rand((0:n2))) # 

  0.113066 seconds (242.06 k allocations: 12.396 MiB)


511109

In [13]:
n2 / ans |> Integer

80649226449312944923625865516066165121534636571013988402916275680391475594995590303139867339021027880416567530699608299655524959846466222965299694593362331883345064161024369750999039300511653259378688

In [14]:
@time memoriezedTurtleHare(n2, 1, rand((0:n2)))

  0.112932 seconds (317.40 k allocations: 16.182 MiB)


511109

In [15]:
n2 / ans |> Integer

80649226449312944923625865516066165121534636571013988402916275680391475594995590303139867339021027880416567530699608299655524959846466222965299694593362331883345064161024369750999039300511653259378688

## Задание 5.

In [16]:
"""
    TurtleHare(n::Int, a::Int=1, x0::Int=1)

n - composite number

Returns: non-trivial divisor of n
"""
function TurtleHare(n::Integer, a::Int=1, x0::Integer=1)
    f(x::Integer=x0) = (x^2 + a) % n; 
    
    Hare = x0;
    Turtle = x0;
    
    while true
        Hare = f(f(Hare));
        Turtle = f(Turtle);
        
        d = gcd(Hare - Turtle, n);
        1 < d && d < n && return d;
        d == n && return "Choose other x0";
    end;
end

TurtleHare

In [17]:
@time TurtleHare(n1, 1, rand((0:n1)))

 37.336471 seconds (12.34 k allocations: 560.026 KiB)


113493047

In [18]:
@time TurtleHare(n2, 1, rand((0:n2)))

  0.013081 seconds (23.29 k allocations: 1.118 MiB)


511109

## Задание 6.

In [19]:
"""
    TurtleAchilles(n::Int, a::Int=1, x0::Int=1)

n - composite number

Returns: non-trivial divisor of n
"""
function TurtleAchilles(n::Integer, a::Int=1, x0::Integer=1)
    f(x::Integer=x0) = (x^2 + a) % n; 
    
    Achilles = x0; k = 0;
    Turtle = x0;
    
    while true
        for j=1:2^a
            Turtle = f(Turtle);
        
            d = gcd(Achilles - Turtle, n);
            1 < d && d < n && return d;
            d == n && return "Choose other x0";
        end;
        
        Achilles = Turtle;
        k += 1;
    end;
end

TurtleAchilles

In [20]:
@time TurtleAchilles(n1, 1, rand((0:n1)))

  5.668158 seconds (28.10 k allocations: 1.411 MiB)


113493047

In [21]:
# @time TurtleAchilles(n2, 1, rand((0:n2)))

## Задание 7. 

In [22]:
n1 = 51353;
n2 = 94657;